# LLM (Deepseek) Cynical Classification


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [47]:
from pprint import pprint
import json

In [4]:
import os
from llama_index.core import (
    Settings, 
    SimpleDirectoryReader, 
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.prompts import PromptTemplate
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [32]:
# ollama = Ollama(model="deepseek-r1:7b", json_mode=True, temperature=0.0)
ollama = Ollama(model="llama3.2:3b-instruct-q8_0", json_mode=True, temperature=0.0)
ollama.system_prompt=(
    "You are a helpful assistant that always responds with valid JSON matching the requested structure."
)


The `CynicalClassification` class has a bool `is_cynical` and `reason` fields.
Adds mft_analysis dictionary to score each relevant moral foundation.
Includes primary_foundation to identify the dominant moral foundation.
Adds foundation_explanation to provide specific context about how the primary foundation manifests in the text.


In [33]:
from pydantic.v1 import BaseModel, Field
from typing import Dict

class CynicalClassification(BaseModel):
    is_cynical: bool
    reason: str
    mft_analysis: Dict[str, float] = Field(
        default_factory=lambda: {
            "authority_subversion": 0.0,   # Degree of authority skepticism (0-1)
            "loyalty_betrayal": 0.0,       # Degree of perceived betrayal (0-1)
            "fairness_cheating": 0.0,      # Degree of perceived unfairness (0-1)
        }
    )
    primary_foundation: str = Field(default="")     # Dominant moral foundation driving cynicism
    foundation_explanation: str = Field(default="") # Explanation of how the primary foundation manifests

In [44]:
CYNICISM_CLASSIFICATION_PROMPT = PromptTemplate(
    template="""
    You are an expert in cynical classification using Moral Foundations Theory (MFT).
    Analyze the text for cynicism through three moral foundations:
    1. Authority/Subversion: Skepticism of institutional power, questioning authority
    2. Loyalty/Betrayal: Sense of institutional or ideological betrayal
    3. Fairness/Cheating: Perceived injustices and inequities

    For each foundation, provide a score from 0.0 (not present) to 1.0 (strongly present).
    Identify the primary moral foundation driving any cynicism.

    Text to analyze: {text}

    Provide your analysis in the following JSON structure. Return only valid JSON:
    {{{{
        "is_cynical": true/false,
        "reason": "explanation for the classification",
        "mft_analysis": {{{{
            "authority_subversion": 0.0,
            "loyalty_betrayal": 0.0,
            "fairness_cheating": 0.0
        }}}},
        "primary_foundation": "name of the dominant foundation",
        "foundation_explanation": "explanation of how the primary foundation manifests"
    }}}}
    """
)


In [45]:
sllm = ollama.as_structured_llm(output_cls=CynicalClassification)

In [51]:
def classify_text_cynicism(text):
    response = sllm.complete(CYNICISM_CLASSIFICATION_PROMPT.format(text=text))
    return json.loads(response.text)

In [52]:
text = "The police are corrupt and they are not there to protect the people."
pprint(classify_text_cynicism(text), indent=2)

{ 'foundation_explanation': 'The primary moral foundation driving cynicism is '
                            'the skepticism towards institutional power, as '
                            "evident in the statement 'they are not there to "
                            "protect the people', which questions the "
                            'authority of the police.',
  'is_cynical': True,
  'mft_analysis': { 'authority_subversion': 1.0,
                    'fairness_cheating': 0.0,
                    'loyalty_betrayal': 0.0},
  'primary_foundation': 'Authority/Subversion',
  'reason': 'The text expresses skepticism towards institutional power, '
            'questioning the authority of the police.'}


In [53]:
text = "The way the adminstration is treating new immigrants is unfair."
pprint(classify_text_cynicism(text), indent=2)

{ 'foundation_explanation': 'The text mentions that the administration is '
                            'treating new immigrants unfairly, indicating a '
                            'perceived injustice and inequity, which is a '
                            'strong manifestation of the Fairness/Cheating '
                            'moral foundation.',
  'is_cynical': True,
  'mft_analysis': { 'authority_subversion': 0.0,
                    'fairness_cheating': 1.0,
                    'loyalty_betrayal': 0.0},
  'primary_foundation': 'Fairness/Cheating',
  'reason': 'The text expresses a sentiment of perceived injustice and '
            'inequity towards new immigrants, indicating a strong presence of '
            'cynicism.'}


In [54]:
text = "Arab Americans are dismayed by the way the administration has betrayed them after the election."
pprint(classify_text_cynicism(text), indent=2)

{ 'foundation_explanation': 'The text expresses skepticism towards '
                            'institutional power, questioning authority in the '
                            "administration's treatment of Gazans after the "
                            'election.',
  'is_cynical': True,
  'mft_analysis': { 'authority_subversion': 0.8,
                    'fairness_cheating': 0.0,
                    'loyalty_betrayal': 0.2},
  'primary_foundation': 'Authority/Subversion',
  'reason': 'The text expresses skepticism towards institutional power, '
            "questioning authority in the administration's treatment of Gazans "
            'after the election.'}
